In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!mkdir tmp

In [ ]:
# Data Load
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
num_hidden = 20
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
hidden = tf.layers.dense(x, units=num_hidden, activation=tf.nn.sigmoid, use_bias=True)
logits = tf.layers.dense(hidden, units=10, activation=None, use_bias=True)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(tf.nn.softmax(logits),1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
batch_size = 128
lr = 0.01
MaxEpochs = 5
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [ ]:
# Checkpoint
saver = tf.train.Saver(max_to_keep=25)

# Train
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(MaxEpochs):
    for step in range(len(mnist.train.images) // batch_size + 1):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
        if step % 200 == 0:
            curr_loss = sess.run(loss, feed_dict={x:mnist.train.images, y:mnist.train.labels})
            curr_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
            print(epoch, step, curr_loss, curr_acc)
            saver.save(sess, "./tmp/model_epoch{0}_step_{1}.ckpt".format(epoch, step))
    print("")

In [ ]:
saver.restore(sess, 'tmp/model_epoch2_step_200.ckpt')
curr_loss = sess.run(loss, feed_dict={x:mnist.train.images, y:mnist.train.labels})
curr_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print(curr_loss, curr_acc)